Αρχικοπιήση και Imports που ειναι απαραιτητα

In [1]:
import json
import os
import sys
import numpy as np
import matplotlib.pyplot as plt


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  #Auto den xreiazetai an exoume polla vms

In [3]:
os.environ.pop('TF_CONFIG', None)

In [4]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

In [5]:

pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

Ειαγωγή dataset (Την γράφουμε τον κώδικα της  εισαγωγής  σε ένα python αρχείο το οποίο θα χρησιμοποιηθεί από όλους τους workers)

In [8]:
%%writefile fashion_mnist_setup.py

import tensorflow as tf
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
 


# load train and test dataset
def fashion_mnist_dataset(batch_size):
  # load dataset
  (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
  # reshape dataset to have a single channel
  trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
  testX = testX.reshape((testX.shape[0], 28, 28, 1))
  # one hot encode target values
  trainY = to_categorical(trainY)
  testY = to_categorical(testY)

  # convert from integers to floats
  train_norm = trainX.astype('float32')
  test_norm = testX.astype('float32')
  # normalize to range 0-1
  train_norm = train_norm / 255.0
  test_norm = test_norm / 255.0
  # return normalized images

  train_dataset = tf.data.Dataset.from_tensor_slices(
      (train_norm, trainY)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset ,train_norm, trainY ,test_norm ,testY
	


def build_and_compile_cnn_model():
  model = Sequential()
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(lr=0.01, momentum=0.9)
  model.summary()
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

Overwriting fashion_mnist_setup.py


Μόνος ένας worker

In [16]:
import fashion_mnist_setup
batch_size = 32
single_worker_dataset ,x_train, y_train , x_test ,y_test = fashion_mnist_setup.fashion_mnist_dataset(batch_size)
single_worker_model =fashion_mnist_setup.build_and_compile_cnn_model()
#single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)
history = single_worker_model.fit(single_worker_dataset, epochs=10, batch_size=32,steps_per_epoch=70, validation_data=(x_test, y_test))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 12544)             0         
                                                                 
 dense_12 (Dense)            (None, 100)               1254500   
                                                                 
 dense_13 (Dense)            (None, 10)                1010      
                                                                 
Total params: 1,256,150
Trainable params: 1,256,150
Non-trainable params: 0
____________________________________________

In [ ]:

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
 
train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
 
r = single_worker_model.fit(train_generator, validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch, epochs=50)

Epoch 1/50
  25/1562 [..............................] - ETA: 8:00 - loss: 1.6131 - accuracy: 0.4412

KeyboardInterrupt: ignored

2 workers

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

gia to deutero

tf_config['task']['index']=1

to trito

tf_config['task']['index']=2

ktl

In [ ]:
json.dumps(tf_config)

'{"cluster": {"worker": ["localhost:12345", "localhost:23456"]}, "task": {"type": "worker", "index": 0}}'

Ekapideysh tou modelou

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()


In [ ]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = cifar_setup.build_and_compile_cnn_model()

In [ ]:
%%writefile main.py

import os
import json

import tensorflow as tf
import cifar_setup

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = cifar_setup.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = cifar_setup.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

Writing main.py


In [ ]:
%%bash
ls *.py

main.py
mnist_setup.py


In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
# first kill any previous runs
%killbgscripts

All background processes were killed.


In [ ]:
%%bash --bg
python main.py &> job_0.log

In [ ]:
import time
time.sleep(10)

In [ ]:
%%bash
cat job_0.log

2022-09-04 15:03:39.917085: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-04 15:03:39.917227: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-04 15:03:39.917243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-09-04 15:03:42.023861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

ta apo kato gia ton worker number 2

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
bash
python main.py

meta kanoume experiment me sharding dld pos na xorisontai ta dta se kommatia

des https://www.tensorflow.org/tutorials/distribute/input#sharding

gia na kaneis xrisi checkpoints h na kaneis save to modelo  kai ta bari xrisimopoies https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb#scrollTo=XVk4ftYx6JAO